In [ ]:
#@title Default title text
import pandas as pd
df=pd.read_csv('fifa19.csv')
df.head()


In [ ]:
df.info()

### **Search for column have unique values to distinguish every player**

In [ ]:
df['ID'].value_counts()

###**Remove some useless columns**

In [ ]:
df=df.drop(columns=['Unnamed: 0','Potential','Real Face','Name','Photo','Nationality','Flag','Club Logo','Special','Loaned From','Preferred Foot','Jersey Number','LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB'],axis=1)
df

# *Remove duplicated rows*

In [ ]:
  df=df.drop_duplicates()

In [ ]:
df.isnull().sum()

In [ ]:
df[df['Body Type'].isna()]

In [ ]:
df['Body Type'].isnull().sum()

In [ ]:
df.dropna(subset=['Body Type'],inplace=True)

In [ ]:
df[['Value']].info()

In [ ]:
df['Value'][0]

In [ ]:
df['Value'][0].split('€')[1][-1:]

In [ ]:
df['Value'][0].split('€')[1][:-1]

In [ ]:
df['M_K']=df['Value'].apply(lambda x : x.split('€')[1][-1:])
df[['M_K']].head()

In [ ]:
df['Value']=df['Value'].apply(lambda x: x.split('€')[1][:-1])
df[['Value']].head()

In [ ]:

df['Value']=pd.to_numeric(df['Value'],errors='coerce')


In [ ]:

df.loc[df['M_K']== 'M','Value(K)'] = df['Value']*1000
df.loc[df['M_K']!= 'M','Value(K)'] = df['Value']


In [ ]:
df=df.drop(columns=['M_K','Value'],axis=1)
df.head()

In [ ]:

df[['Wage']].info()


In [ ]:
df['Wage'][0]

In [ ]:
df['Wage'][0].split('€')[1][:-1]

In [ ]:
df['Wage(K)']=df['Wage'].apply(lambda x: x.split('€')[1][:-1])
df['Wage(K)']=pd.to_numeric(df['Wage(K)'],errors='coerce')
df=df.drop(columns='Wage',axis=1)
df[['Wage(K)']]

In [ ]:

df[['Release Clause']].info()


In [ ]:
df['Release Clause'][0]

In [ ]:
df['Release Clause'][0].split('€')[1][:-1]

In [ ]:
df['Release Clause'][0].split('€')[1][-1:]

In [ ]:
df['Release Clause'].fillna(df['Release Clause'].mode()[0],inplace=True)

In [ ]:

df['M_K']=df['Release Clause'].apply(lambda x : x.split('€')[1][-1:])
df['Release Clause']=df['Release Clause'].apply(lambda x : x.split('€')[1][:-1])
df['Release Clause']=pd.to_numeric(df['Release Clause'],errors='coerce')
df.loc[df['M_K']== 'M','Release Clause(K)'] = df['Release Clause']*1000
df.loc[df['M_K']!= 'M','Release Clause(K)'] = df['Release Clause']
df[['Release Clause(K)']]

In [ ]:
df=df.drop(columns=['M_K','Release Clause'],axis=1)
df.head()

In [ ]:
df[['Weight']].info()

In [ ]:
df['Weight'].fillna(df['Weight'].mode()[0], inplace=True)

In [ ]:
df['Weight'][0]

In [ ]:
df['Weight'][0].split('lbs')[0]

In [ ]:
df['Weight(kg)']=df['Weight'].apply(lambda x: x.split('lbs')[0])
df['Weight(kg)']=pd.to_numeric(df['Weight(kg)'],errors='coerce')
df['Weight(kg)']=df['Weight(kg)']*0.45359237
df[['Weight(kg)']]

In [ ]:
df=df.drop(columns=['Weight'],axis=1)

In [ ]:
df[['Height']].info()

In [ ]:
df['Height'].fillna(df['Height'].mode()[0], inplace=True)

In [ ]:
df['Height'][0]

In [ ]:
df['Height'][0].split("'")[0]

In [ ]:
df['Height'][0].split("'")[1]

In [ ]:
df['inch']=df['Height'].apply(lambda x: x.split("'")[1])
df['inch']=pd.to_numeric(df['inch'],errors='coerce',downcast='float')
df[['inch']]

In [ ]:
df['feet']=df['Height'].apply(lambda x: x.split("'")[0])
df['feet']=pd.to_numeric(df['feet'],errors='coerce',downcast='float')
df[['feet']]

In [ ]:
df['Height(cm)']=(df['inch']*2.54)+(df['feet']*30.48)
df[['Height(cm)']]

In [ ]:
df=df.drop(columns=['feet','inch','Height'],axis=1)



In [ ]:
df.head()

In [ ]:
df[['Work Rate']].info()

In [ ]:
df['Work Rate'].unique()

In [ ]:
df.dropna(subset=['Work Rate'],inplace=True)


In [ ]:
df['Work Rate'][0].split('/ ')[0]

In [ ]:
df['Attack Work Rate']=df['Work Rate'].apply(lambda x: x.split('/ ')[0])
df['Defence Work Rate']=df['Work Rate'].apply(lambda x: x.split('/ ')[1])
df.head()

In [ ]:

W_Dec={'High':3,'Medium':2,'Low':1}
df['Attack Work Rate']=df['Attack Work Rate'].map(W_Dec)
df['Defence Work Rate']=df['Defence Work Rate'].map(W_Dec)
df.head()

In [ ]:
df=df.drop('Work Rate',axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.impute import KNNImputer
imputer=KNNImputer()
cols=['Value(K)','Wage(K)']
for col in cols:
  df[col]=imputer.fit_transform(df[[col]])
df.isnull().sum()

In [ ]:
df[['Joined']].info()

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='most_frequent')
df['Joined']=imputer.fit_transform(df[['Joined']])


In [ ]:
from datetime import datetime
df['Joined']=df['Joined'].apply(lambda x: datetime.strptime(x ,'%b %d, %Y') )
df[['Joined']]

In [ ]:
df[['Contract Valid Until']].info()

In [ ]:
df['Contract Valid Until']=pd.to_datetime(df['Contract Valid Until'],format='%Y',errors='coerce')
df['Contract Valid Until'].fillna(df['Contract Valid Until'].mean(),inplace=True)

In [ ]:
import numpy as np
df['Duration']=(df['Contract Valid Until']-df['Joined'])/np.timedelta64(1,'Y')
df[['Duration']]

In [ ]:
df[['Position']].info()

In [ ]:
df['Position'].isnull().sum()

In [ ]:
df['Position'].fillna(df['Position'].mode()[0],inplace=True)

In [ ]:
df[['Club']].info()

In [ ]:
df['Club'].unique()

In [ ]:
df.dropna(subset=['Club'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
cols=['Attack Work Rate','Defence Work Rate','ID','Age','Overall']
for col in cols:
  df[col]=df[col].astype(float)

In [ ]:
df.describe()

In [ ]:
df['Duration']=df['Duration'].apply(lambda x:abs(x))

In [ ]:
cols=['Value(K)','Wage(K)','Duration','Release Clause(K)','Age']
for Col in cols:
  Q1= df[Col].quantile(0.25)
  Q3= df[Col].quantile(0.75)
  iqr=Q3-Q1
  lower_limit = Q1 - (1.5 * iqr)
  upper_limit = Q3 + (1.5 * iqr)
  x=df.loc[df[Col]>upper_limit]
  y=df.loc[df[Col]<lower_limit]
  df.loc[x.index,Col]=df.loc[x.index,Col]-(1.5*iqr)
  df.loc[y.index,Col]=df.loc[y.index,Col]+(1.5*iqr)

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':[10,10]},font_scale=1)

In [ ]:
sns.lmplot(x='Height(cm)',y='Weight(kg)',data=df)

In [ ]:
sns.lmplot(x='Wage(K)',y='Duration',data=df)

In [ ]:
sns.lmplot(x='Wage(K)',y='Value(K)',data=df)

In [ ]:
sns.lmplot(x='Wage(K)',y='Release Clause(K)',data=df)

In [ ]:
sns.lmplot(x='Wage(K)',y='Age',data=df)

In [ ]:
sns.catplot(y='Club',x='Wage(K)',data=df,kind='bar')

In [ ]:
sns.lmplot(x='Value(K)',y='Age',data=df)

In [ ]:
sns.lmplot(x='Value(K)',y='International Reputation',data=df)

In [ ]:
sns.lmplot(x='Value(K)',y='Weak Foot',data=df)

In [ ]:
sns.lmplot(x='Value(K)',y='Skill Moves',data=df)

In [ ]:
sns.lmplot(x='Release Clause(K)', y='Duration', data=df)

In [ ]:
sns.lmplot(x='Release Clause(K)', y='Wage(K)', data=df)

In [ ]:
sns.lmplot(x='Release Clause(K)', y='Value(K)', data=df)

In [ ]:
sns.lmplot(x='Overall', y='International Reputation', data=df)

In [ ]:
sns.lmplot(x='Overall', y='Value(K)', data=df)

In [ ]:
sns.lmplot(x='Overall', y='Age', data=df)

In [ ]:
sns.set(rc={'figure.figsize':[20,20]},font_scale=0.8)
sns.catplot(x='Body Type',y='Overall',data=df,kind='strip',order=["Normal", "Lean","Stocky"])

In [ ]:
sns.catplot(y='Position',x='Overall',data=df,kind='strip')

In [ ]:
sns.catplot(y='Position',x='Value(K)',data=df,kind='strip')

In [ ]:
df.info()

In [ ]:
x=df.drop(columns=['ID','Duration','Wage(K)','Value(K)','Release Clause(K)','Weight(kg)','Height(cm)','Body Type','Position','Joined','Contract Valid Until'],axis=1)
x=pd.get_dummies(x,columns=['Club'])
y=df['Value(K)']

In [ ]:
x.info()

In [ ]:
x=x.astype('float64')

In [ ]:
x.info()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
y_train.value_counts()

In [ ]:
len(y_train)/len(y)

In [ ]:
len(y_test)/len(y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_train

In [ ]:
x=df[['Club','Value(K)','Release Clause(K)','Age']]
x=pd.get_dummies(x,columns=['Club'])
y=df['Wage(K)']


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
y_train.value_counts()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_train

In [ ]:
x=df[['Club','Value(K)','Wage(K)']]
x=pd.get_dummies(x,columns=['Club'])
y=df['Release Clause(K)']


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
y_train.value_counts()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_train

In [ ]:
x=df.drop(columns=['ID','Overall','Duration','Wage(K)','Value(K)','Release Clause(K)','Weight(kg)','Height(cm)','Position','Joined','Contract Valid Until','Age','Body Type','International Reputation','Club'],axis=1)
y=df['Position']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
y_train.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_train

In [ ]:
x=df[['Weight(kg)','Height(cm)']]
y=df['Body Type']
y.value_counts()

In [ ]:
lst=['Courtois','PLAYER_BODY_TYPE_25','Neymar','C. Ronaldo','Shaqiri','Messi','Akinfenwa']
for item in lst:
  df.loc[df['Body Type']== item,'Body Type'] = df['Body Type'].mode()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_train

In [ ]:
x=df.drop(columns=['ID','Joined','Position','Contract Valid Until','Duration','Overall','Release Clause(K)','Wage(K)','Body Type'])
x=pd.get_dummies(x,columns=['Club'])
y=df['Overall']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
y_train.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_train